In [1]:
import csv,mechanize,re,time,json
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import pandas as pd

Get the episode ratings and average to season/year

In [3]:
from imdb import IMDb

In [4]:
ia = IMDb()

In [5]:
shows = pd.read_csv('./data/gglobes_data_wikipedia_1972to2020.csv')


In [6]:
shows.show.nunique()

394

In [13]:
#Make a list of cases to correct where the first returned result is not the right one

corrections_dict = {
    'The Flip Wilson Show': '0065294',
    'America': '0185051',
    'Kung Fu': '0068093',
    'Police Story': '0069620',
    'Charlie\'s Angels': '0073972',
    'Battlestar Galactica': '0076984',
    'Three\'s Company':'0075596',
    'Alice': '0073955',
    '60 Minutes[a]': '0123338',
    'The Associates': '0078563',
    'From Here to Eternity': '0078616',
    'Soap': '0075584',
    'Dynasty': '0081856',
    'Flamingo Road': '0081863',
    'Private Benjamin': '0081918',
    'One Day at a Time': '0072554',
    'The Equalizer': '0088513',
    'Growing Pains': '0088527',
    'Beauty and the Beast': '0092319',
    'Dear John': '0094447',
    'Wiseguy': '0092484',
    'Pros and Cons': '0101173',
    'Homefront': '0101121',
    'Sisters': '0101199',
    'The Guardian': '0285370',
    'Boss': '1833285',
    'Necessary Roughness': '1657505',
    'Graves': '4866982',
    'The Odd Couple': '0065329',
    'Mission: Impossible': '0060009',
    'The Blue Knight': '0072478',
    'All\'s Fair': '0073956',
    'Holocaust': '0077025',
    'The Paper Chase': '0077058',
    'Wheels': '0077099',
    'House Calls': '0078626',
    'Fame': '0083412',
    'Nine to Five': '0083377',
    'Buffalo Bill': '0084992',
    'Nothing Sacred': '0118411',
    'House of Lies': '1797404',
    'Cannon': '0066636',
    'Lotsa Luck': '0069603',
    '9 to 5': '0083377',
    'Law and Order: Special Victims Unit': '0203259',
    'Empire': '3228904'
}

show_ids_dict = {}

for show in shows.show.unique():
    
    if show in corrections_dict.keys():
        show_ids_dict[show] = corrections_dict[show]
    else:
        res = ia.search_movie(show)
        #print(f"{show}: {res[0]['long imdb title']}")
        show_ids_dict[show] = res[0].movieID

Get the season ratings and years for all the shows

In [40]:
tmp = list(show_ids_dict.items())
len(tmp)

394

In [ ]:
show_ratings = []

In [ ]:
for show, show_id in show_ids_dict.items():
#for (show, show_id) in tmp[350:]:
    #These is actually a TV movie - no episodes
    #General Hospital, Carson are a series but gives an error, probably because too many episodes
    if show in ['The Homecoming: A Christmas Story','America','The Scarlett OHara War','Moviola',
                'General Hospital','The Tonight Show Starring Johnny Carson']:
        continue
    
    eps = ia.get_movie_episodes(show_id)

    for season in eps['data']['episodes']:
        #Mr Robot has a -1 season 'Virtual Reality Experience'
        if season < 1:
            continue

        print(show, season)
        first_ep_id = list(eps['data']['episodes'][season].keys())[0]
        if 'year' in eps['data']['episodes'][season][first_ep_id].keys():
            season_start_year = eps['data']['episodes'][season][first_ep_id]['year']

        av_rating, av_num_votes, num_eps = 0., 0., 0
        for episode in eps['data']['episodes'][season].values():
            #exclude episodes that have not aired and been reviewed yet
            if 'rating' not in episode.keys() or 'votes' not in episode.keys() or 'year' not in episode.keys():
                continue
            #print(f"{episode['smart canonical title']} ({episode['year']}): {episode['rating']:.1f} from {episode['votes']} votes")
            if episode['rating'] < 1 or episode['rating'] > 10:
                print('strange rating', episode['rating'], episode['votes'])
            av_rating += episode['rating']
            num_eps += 1
            av_num_votes += episode['votes']
            
        if num_eps > 0:
            av_rating = np.round(av_rating/num_eps,2)
            av_num_votes = np.round(av_num_votes/num_eps,0)

            #print(f"==== Season {season} average rating = {av_rating:.2f}")
            show_ratings.append([show, season, season_start_year, num_eps, av_rating, av_num_votes])

In [75]:
with open('./data/imdb_data_gglobesnominees_wikipedianames_1972to2020.csv','w') as f:
    mywriter = csv.writer(f)
    mywriter.writerow(['show', 'season', 'season_start_year', 'num_eps', 'av_rating', 'av_num_votes'])
    for row in show_ratings:
        mywriter.writerow(row)

Do for 2021 pending

In [12]:
shows = pd.read_csv('gglobes_data_2021pending.csv')

shows.show.unique()

array(['Ozark', 'The Crown', 'Better Call Saul', 'Hunters', 'Perry Mason',
       'Black Monday', 'The Great', "Schitt's Creek", 'Ted Lasso', 'Ramy',
       'Killing Eve', 'Ratched', 'Emily in Paris', 'The Flight Attendant',
       "Zoey's Extraordinary Playlist", 'Lovecraft Country',
       'The Mandalorian'], dtype=object)

In [11]:
corrections_dict = {
    'Perry Mason': '2077823'
}
show_ids_dict = {}

for show in shows.show.unique():
    
    if show in corrections_dict.keys():
        show_ids_dict[show] = corrections_dict[show]
    else:
        res = ia.search_movie(show)
        #print(f"{show}: {res[0]['long imdb title']}")
        show_ids_dict[show] = res[0].movieID

In [ ]:
show_ratings = []

for show, show_id in show_ids_dict.items():
    
    eps = ia.get_movie_episodes(show_id)

    for season in eps['data']['episodes']:
        #Mr Robot has a -1 season 'Virtual Reality Experience'
        if season < 1:
            continue

        print(show, season)
        first_ep_id = list(eps['data']['episodes'][season].keys())[0]
        if 'year' in eps['data']['episodes'][season][first_ep_id].keys():
            season_start_year = eps['data']['episodes'][season][first_ep_id]['year']

        av_rating, av_num_votes, num_eps = 0., 0., 0
        for episode in eps['data']['episodes'][season].values():
            #exclude episodes that have not aired and been reviewed yet
            if 'rating' not in episode.keys() or 'votes' not in episode.keys() or 'year' not in episode.keys():
                continue
            #print(f"{episode['smart canonical title']} ({episode['year']}): {episode['rating']:.1f} from {episode['votes']} votes")
            if episode['rating'] < 1 or episode['rating'] > 10:
                print('strange rating', episode['rating'], episode['votes'])
            av_rating += episode['rating']
            num_eps += 1
            av_num_votes += episode['votes']
            
        if num_eps > 0:
            av_rating = np.round(av_rating/num_eps,2)
            av_num_votes = np.round(av_num_votes/num_eps,0)

            #print(f"==== Season {season} average rating = {av_rating:.2f}")
            show_ratings.append([show, season, season_start_year, num_eps, av_rating, av_num_votes])

In [14]:
with open('imdb_data_gglobesnominees_2021pending.csv','w') as f:
    mywriter = csv.writer(f)
    mywriter.writerow(['show', 'season', 'season_start_year', 'num_eps', 'av_rating', 'av_num_votes'])
    for row in show_ratings:
        mywriter.writerow(row)